In [1]:
#pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg2

# spajanje na posgres bazu
conn = psycopg2.connect("dbname=postgres user=postgres password=d1nk0sql")

In [24]:
import pandas as pd

# prepare a cursor object using cursor() method
cursor = conn.cursor()

# definiranje sql selecta
sql = "select * from dinko_proba.transactions"

# okidamo kursor
cursor.execute(sql)
data = cursor.fetchall()

# nazivi kolona
col_names = [i[0] for i in cursor.description]

# poclacenje podataka u dataframe
df = pd.DataFrame(data, columns=col_names)

print(df.head())

print(' ')

print(df.dtypes)

      tnxdate    product tnxamount
0  2020-01-01  Chair         23.01
1  2020-01-01  Chair         32.02
2  2020-01-02  Chair         56.04
3  2020-01-03  Chair         34.37
4  2020-01-03  Chair         65.82
 
tnxdate      object
product      object
tnxamount    object
dtype: object


# Create table in database

In [19]:
# get column names
columnName = list(df.columns.values)

print(columnName)

['tnxdate', 'product', 'tnxamount']


In [20]:
# find column data type
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

columnDataType = getColumnDtypes(df.dtypes)

print(columnDataType)

['varchar', 'varchar', 'varchar']


In [25]:
# create SQL statment
createTableStatement = 'CREATE TABLE IF NOT EXISTS dinko_proba.transactions_xx ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

print(createTableStatement)

CREATE TABLE IF NOT EXISTS dinko_proba.transactions_xx (
tnxdate varchar,
product varchar,
tnxamount varchar );


In [26]:
# create table
cursor.execute(createTableStatement)

conn.commit()

In [15]:
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [27]:
#write dataframe to database
execute_many(conn, df, 'dinko_proba.transactions_xx')

execute_many() done


In [28]:
conn.close()